# Importing datasets
* Copy the datasets from https://www.kaggle.com/azathoth42/myanimelist into `data/imported_datasets/azathoth42`
* Run the notebook

In [ ]:
# TODO this is wildly out of date

In [1]:
import os
import pickle

import pandas as pd

In [2]:
source_dir = "../../data/imported_datasets/azathoth42/"

In [3]:
outdir = "../../data/raw_data"
if not os.path.exists(outdir):
    os.mkdir(outdir)

## Get anime facts

In [4]:
anime = pd.read_csv(os.path.join(source_dir, "AnimeList.csv"))

In [5]:
anime = anime[["anime_id", "title", "genre", "type", "related"]].rename(
    {"genre": "genres", "related": "related_anime", "type": "medium"}, axis=1
)

In [6]:
def process_related(related):
    related = eval(related)
    return [
        {"anime_id": entry["mal_id"], "relation": relation.lower()}
        for relation in related
        for entry in related[relation]
        if entry["type"] == "anime"
    ]

In [7]:
anime["related_anime"] = anime["related_anime"].apply(process_related)

In [11]:
anime["recommendations"] = [[] for _ in range(len(anime))]

In [13]:
anime = anime.sort_values(by="anime_id")

In [14]:
anime.to_csv(os.path.join(outdir, "anime.csv"), index=False)

## Get users

In [10]:
users = pd.read_csv(os.path.join(source_dir, "UserList.csv"))

In [11]:
users[["username"]].to_csv(os.path.join(outdir, "usernames.csv"), index=False)

## Get User Lists

In [ ]:
# This may take several minutes to finish
anime_lists = pd.read_csv(os.path.join(source_dir, "UserAnimeList.csv"))

In [ ]:
anime_lists = anime_lists[["username", "anime_id", "my_score"]]
anime_lists = anime_lists.loc[lambda x: x["my_score"] != 0].reset_index(drop=True)

In [ ]:
anime_lists.to_pickle(os.path.join(outdir, "user_anime_lists.pkl"))

In [ ]:
# This may take several minutes to finish
anime_lists.to_csv(os.path.join(outdir, "user_anime_lists.csv"), index=False)

## Write source

In [ ]:
with open(os.path.join(outdir, "source.txt"), "w") as f:
    f.write("Dataset available at https://www.kaggle.com/azathoth42/myanimelist")